In [1]:
from dotenv import load_dotenv

load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.tools import QueryEngineTool

# settings
import GPUtil
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.mistralai import MistralAI
from llama_index.core import SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path
from utils import get_doc_tools
from loguru import logger

device = "cuda" if len(GPUtil.getAvailable()) >= 1 else "cpu"

llm = MistralAI(
    api_key="jiSxvwweunDg9qY8LasnngBrqPVaPMGb",
    temperature=0.1,
)

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2", device=device
)
Settings.llm = llm
Settings.embed_model = embed_model

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Prepare Tools

In [2]:
sources = [
    ("./data/COT.csv", "Coach On Tap platform"),
    ("./data/ANZ.pdf", "ANZ Bank"),
]
source_to_tools_dict = {}
for source, desc in sources:
    print(f"Getting tools for source: {source}")
    vector_tool, summary_tool = get_doc_tools(source, Path(source).stem, desc)
    source_to_tools_dict[source] = [vector_tool, summary_tool]

# Create all tools
all_tools = [t for s, _ in sources for t in source_to_tools_dict[s]]
for i in all_tools:
    print(i.metadata)

Getting tools for source: ./data/COT.csv
Getting tools for source: ./data/ANZ.pdf
ToolMetadata(description='Useful if you want to get a summary of Coach On Tap platform.', name='cot_summary_tool', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(description='A RAG engine with some facts about Coach On Tap platform.', name='cot_qna_tool', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(description='Useful if you want to get a summary of ANZ Bank.', name='anz_summary_tool', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(description='A RAG engine with some facts about ANZ Bank.', name='anz_qna_tool', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)


## Prepare Agent

In [3]:
agent = ReActAgent.from_tools(all_tools, verbose=False)

## Chat

In [5]:
stream_response = agent.stream_chat(
    "Calculate sum of the coaching platform's profit in 3 months. Each month platform earns 2$"
)

stream_response.chat_stream

<generator object ReActAgentWorker._add_back_chunk_to_stream.<locals>.gen at 0x31c4efc60>

In [ ]:
response = agent.chat(
    "How do I know whether my ANZ Digital Key is setup to receive notifications or to use a QRcode?"
)

print(response)
logger.info(str(response))

In [ ]:
response = agent.chat(
    "Give me summary of ANZ DK"
)

print(response)
logger.info(str(response))